In [ ]:
>>> fizzbuzz = lambda x: "Fizzbuzz" if x%3 == 0 and x%5 == 0 else "Fizz" if x%3 == 0 else "Buzz" if x%5 == 0 else str(x)
>>> print([fizzbuzz(i) for i in range(1,101)])

In [ ]:
>>> import execnet
>>> def square(channel):
...     while not channel.isclosed():
...         number = channel.receive()
...         number_squared = number**2
...         channel.send(number_squared)
>>> gateway = execnet.makegateway()
>>> channel = gateway.remote_exec(square)
>>> for i in range(10):
...     channel.send(i)
...     i_squared = channel.receive()
...     print(f"{i} squared is {i_squared}")
>>> gateway.exit()

In [ ]:
>>> import multiprocessing
>>> def square_mp(in_queue, out_queue):
...     while(True):
...         n = in_queue.get()
...         n_squared = n**2
...         out_queue.put(n_squared)

>>> if __name__ == '__main__':
...     in_queue = multiprocessing.Queue()
...     out_queue = multiprocessing.Queue()
...     process = multiprocessing.Process(target=square_mp, args=(in_queue, out_queue))
...     process.start()
...     for i in range(10):
...         in_queue.put(i)
...         i_squared = out_queue.get()
...         print(f"{i} squared is {i_squared}")
...     process.terminate()


In [ ]:
>>> import threading
>>> import queue
>>> in_queue = queue.Queue()
>>> out_queue = queue.Queue()
>>> def square_threading():
...     while True:
...         n = in_queue.get()
...         if n == 'STOP':
...             return
...         n_squared = n**2
...         out_queue.put(n_squared)

>>> thread = threading.Thread(target=square_threading)
>>> thread.start()
>>> for i in range(10):
...     in_queue.put(i)
...     i_squared = out_queue.get()
...     print(f"{i} squared is {i_squared}")
>>> in_queue.put('STOP')
>>> thread.join()

In [ ]:
%%writefile exercise8.py
>>> import argparse
>>> parser = argparse.ArgumentParser(description="Interpret a Boolean flag.")
>>> parser.add_argument('--flag', dest='flag', action='store_true', help='Set the flag value to True.')
>>> arguments = parser.parse_args()
>>> print(f"The flag's value is {arguments.flag}")

In [ ]:
%%writefile exercise9.py
>>> import argparse
>>> parser = argparse.ArgumentParser(description="Interpret positional arguments.")
>>> parser.add_argument('source', action='store', help='The source of an operation.')
>>> parser.add_argument('dest', action='store', help='The destination of the operation.')
>>> arguments = parser.parse_args()
>>> print(f"Picasso will cycle from {arguments.source} to {arguments.dest}")

In [ ]:
%%writefile activity2.py
>>> import argparse
>>> parser = argparse.ArgumentParser(description="Play the game of FizzBuzz.")
>>> parser.add_argument("fizz", action="store", help="Number to replace multiples of by 'Fizz'.", type=int)
>>> parser.add_argument("buzz", action="store", help="Number to replace multiples of by 'Buzz'.", type=int)
>>> arguments = parser.parse_args()
>>> fizz = arguments.fizz
>>> buzz = arguments.buzz
>>> fizzbuzz = lambda x: "Fizzbuzz" if x%fizz == 0 and x%buzz == 0 else "Fizz" if x%fizz == 0 else "Buzz" if x%buzz == 0 else str(x)
>>> print([fizzbuzz(i) for i in range(1,101)])

In [ ]:
>>> import timeit
>>> class PrimesBelow:
...     def __init__(self, bound):
...         self.candidate_numbers = list(range(2,bound))
...     def __iter__(self):
...         return self
...     def __next__(self):
...         if len(self.candidate_numbers) == 0:
...             raise StopIteration
...         next_prime = self.candidate_numbers[0]
...         self.candidate_numbers = [x for x in self.candidate_numbers if x % next_prime != 0]
...         return next_prime
>>> print(timeit.timeit('list(PrimesBelow(1000))', setup='from __main__ import PrimesBelow', number=10000))

In [ ]:
%%writefile sieve_module.pyx
>>> class PrimesBelow:
...     def __init__(self, bound):
...         self.candidate_numbers = list(range(2,bound))
...     def __iter__(self):
...         return self
...     def __next__(self):
...         if len(self.candidate_numbers) == 0:
...             raise StopIteration
...         next_prime = self.candidate_numbers[0]
...         self.candidate_numbers = [x for x in self.candidate_numbers if x % next_prime != 0]
...         return next_prime

In [ ]:
%%writefile setup.py
>>> from distutils.core import setup
>>> from Cython.Build import cythonize
>>> setup(
...     ext_modules = cythonize("sieve_module.pyx")
... )

In [ ]:
import timeit

print(timeit.timeit('list(PrimesBelow(1000))', setup='from sieve_module import PrimesBelow', number=10000))

In [ ]:
>>> class Primes:
...     def __init__(self):
...         self.current = 2
...     def __iter__(self):
...         return self
...     def __next__(self):
...         while True:
...             current = self.current
...             square_root = int(current ** 0.5)
...             is_prime = True
...             if square_root >= 2:
...                 for i in range(2, square_root + 1):
...                     if current % i == 0:
...                         is_prime = False
...                         break
...             self.current += 1
...             if is_prime:
...                 return current

In [ ]:
>>> import cProfile
>>> import itertools
>>> cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes())]')


In [ ]:
>>> class Primes2:
...     def __init__(self):
...         self.known_primes=[]
...         self.current=2
...     def __iter__(self):
...         return self
...     def __next__(self):
...         while True:
...             current = self.current
...             prime_factors = [p for p in self.known_primes if current % p == 0]
...             self.current += 1
...             if len(prime_factors) == 0:
...                 self.known_primes.append(current)
...                 return current
>>> cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes2())]')


In [ ]:
>>> class Primes3:
...     def __init__(self):
...         self.known_primes=[]
...         self.current=2
...     def __iter__(self):
...         return self
...     def __next__(self):
...         while True:
...             current = self.current
...             sqrt_current = int(current**0.5)
...             potential_factors = itertools.takewhile(lambda x: x < sqrt_current, self.known_primes)
...             prime_factors = [p for p in potential_factors if current % p == 0]
...             self.current += 1
...             if len(prime_factors) == 0:
...                 self.known_primes.append(current)
...                 return current

>>> cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes3())]')

In [ ]:
>>> class Primes4:
...     def __init__(self):
...         self.known_primes=[]
...         self.current=2
...     def __iter__(self):
...         return self
...     def __next__(self):
...         while True:
...             current = self.current
...             sqrt_current = int(current**0.5)
...             potential_factors = itertools.takewhile(lambda x: x < sqrt_current, self.known_primes)
...             is_prime = True
...             for p in potential_factors:
...                 if current % p == 0:
...                     is_prime = False
...                     break
...             self.current += 1
...             if is_prime == True:
...                 self.known_primes.append(current)
...                 return current

>>> cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000, Primes4())]')

In [ ]:
>>> cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000000, Primes())]')
>>> cProfile.run('[p for p in itertools.takewhile(lambda x: x<10000000, Primes4())]')
